# EDA of data


In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Sort out columns and rows that are not neccesary

In [2]:
url = "https://archivo.datos.cdmx.gob.mx/fiscalia-general-de-justicia/victimas-en-carpetas-de-investigacion-fgj/da_victimas_completa_marzo_2023.csv"
data_cdmx = pd.read_csv('da_victimas_completa_marzo_2023.csv')

In [3]:
data_cdmx.head()

,idCarpeta,Año_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,...,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,municipio_hechos,colonia_datos,fgj_colonia_registro,latitud,longitud
0,8324429.0,2019,Enero,2019-01-04,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,...,Agosto,2018-08-29,12:00:00,12:19:00,ALVARO OBREGON,NaN,GUADALUPE INN,GUADALUPE INN,19.36125,-99.18314
1,8324430.0,2019,Enero,2019-01-04,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2018-12-15,15:00:00,12:20:00,AZCAPOTZALCO,NaN,VICTORIA DE LAS DEMOCRACIAS,VICTORIA DE LAS DEMOCRACIAS,19.47181,-99.16458
2,8324431.0,2019,Enero,2019-01-04,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,...,Diciembre,2018-12-22,15:30:00,12:23:00,COYOACAN,NaN,COPILCO EL BAJO,COPILCO UNIVERSIDAD ISSSTE,19.33797,-99.18611
3,8324435.0,2019,Enero,2019-01-04,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,...,Enero,2019-01-04,06:00:00,12:27:00,IZTACALCO,NaN,PANTITLAN V,AGRÍCOLA PANTITLAN,19.40327,-99.05983
4,8324438.0,2019,Enero,2019-01-04,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,VICTIMA,...,Enero,2019-01-03,20:00:00,12:35:00,IZTAPALAPA,NaN,LAS AMERICAS (U HAB),PROGRESISTA,19.35480,-99.06324


In [4]:
len(data_cdmx)

1038430

In [5]:
#sort out delicts that happened outside of cdmx
boolean_mask = data_cdmx['municipio_hechos'].isnull()
data = data_cdmx[boolean_mask].reset_index(drop=True)

data.shape

(1028246, 22)

In [6]:
#sort out crimes that were not commited against persons but companies
boolean_mask = data['TipoPersona'] == 'FISICA'
data = data[boolean_mask].reset_index(drop=True)

In [7]:
data.shape

(819283, 22)

In [8]:
#Which columns can we ommit?


columns_to_drop = ['idCarpeta', 'Año_inicio', 'Mes_inicio', 'FechaInicio', 
                   'CalidadJuridica', 'Mes_hecho',  'HoraInicio', 'alcaldia_hechos',
                  'fgj_colonia_registro', 'municipio_hechos']

data = data.drop(columns_to_drop, axis=1)

In [9]:
data.TipoPersona.value_counts()

FISICA    819283
Name: TipoPersona, dtype: int64

In [10]:
data.shape

(819283, 12)

## Make delict categrories

In [11]:
data.Delito

0                                                    FRAUDE
1         PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...
2         ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...
3         ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...
4                         ROBO DE MOTOCICLETA SIN VIOLENCIA
                                ...                        
819278    ROBO S/V DENTRO DE NEGOCIOS, AUTOSERVICIOS, CO...
819279                                   VIOLENCIA FAMILIAR
819280          ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO
819281                                      ROBO DE OBJETOS
819282                                   VIOLENCIA FAMILIAR
Name: Delito, Length: 819283, dtype: object

In [36]:
#dataset to clasify according to new defined categories
new_cats = pd.read_csv('new_categories4.csv')


In [37]:
new_cats

,Unnamed: 0,Unnamed: 1
0,VIOLENCIA FAMILIAR,domestic violence
1,AMENAZAS,threats
2,TENTATIVA DE EXTORSION,threats
3,DISCRIMINACION,threats
4,NEGACION DEL SERVICIO PUBLICO,threats
...,...,...
308,"PANDILLA, ASOCIACIÓN DELICTUOSA Y DELINCUENCIA...",ignore
309,OTROS DELITOS,ignore
310,PERDIDA DE LA VIDA POR ENVENENAMIENTO,ignore
311,UTILIZACION INDEBIDA DE LA VIA PUBLICA,ignore


In [38]:
#renaming of the columns
new_cats.rename(columns={"Unnamed: 0": "Delito", "Unnamed: 1": "new_cats"},inplace=True)

In [39]:
new_cats[new_cats['Delito']=='ROBO DE MOTOCICLETA SIN VIOLENCIA']

,Delito,new_cats
110,ROBO DE MOTOCICLETA SIN VIOLENCIA,robbery without violence


In [40]:
data_new_cats = pd.merge(data, new_cats,how='left', on='Delito')
data_new_cats

,Delito,Categoria,Sexo,Edad,TipoPersona,competencia,Año_hecho,FechaHecho,HoraHecho,colonia_datos,latitud,longitud,new_cats
0,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,FUERO COMUN,2018.0,2018-08-29,12:00:00,GUADALUPE INN,19.361250,-99.183140,fraud
1,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,FUERO COMUN,2018.0,2018-12-15,15:00:00,VICTORIA DE LAS DEMOCRACIAS,19.471810,-99.164580,fraud
2,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,FUERO COMUN,2018.0,2018-12-22,15:30:00,COPILCO EL BAJO,19.337970,-99.186110,robbery with violence
3,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,FUERO COMUN,2019.0,2019-01-04,06:00:00,PANTITLAN V,19.403270,-99.059830,robbery without violence
4,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,FUERO COMUN,2019.0,2019-01-03,20:00:00,LAS AMERICAS (U HAB),19.354800,-99.063240,robbery without violence
...,...,...,...,...,...,...,...,...,...,...,...,...,...
820140,"ROBO S/V DENTRO DE NEGOCIOS, AUTOSERVICIOS, CO...",DELITO DE BAJO IMPACTO,Femenino,69.0,FISICA,FUERO COMUN,2023.0,2023-03-31,07:30:00,DEL VALLE VII,19.372612,-99.174058,robbery without violence
820141,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Femenino,17.0,FISICA,FUERO COMUN,2023.0,2023-03-30,22:00:00,MIGUEL DE LA MADRID HURTADO,19.347189,-99.003079,domestic violence
820142,ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO,DELITO DE BAJO IMPACTO,Femenino,43.0,FISICA,FUERO COMUN,2023.0,2023-03-31,07:00:00,LOMAS DE SANTA FE II,19.369135,-99.260935,robbery without violence
820143,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,Femenino,39.0,FISICA,FUERO COMUN,2023.0,2023-03-31,10:01:00,SAN PEDRO MARTIR (PBLO),19.267265,-99.175255,robbery without violence


In [41]:
#showing all raws have a new category
data_new_cats[data_new_cats['new_cats'].isnull()].Delito.unique()

array([], dtype=object)

In [42]:
#droping the raws with 'ignore' category
boolean_mask = data_new_cats['new_cats'] != 'ignore'
data_new_cats = data_new_cats[boolean_mask].reset_index(drop=True)

In [43]:
#dataset with new categories ready
data_new_cats

,Delito,Categoria,Sexo,Edad,TipoPersona,competencia,Año_hecho,FechaHecho,HoraHecho,colonia_datos,latitud,longitud,new_cats
0,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,FUERO COMUN,2018.0,2018-08-29,12:00:00,GUADALUPE INN,19.361250,-99.183140,fraud
1,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,FUERO COMUN,2018.0,2018-12-15,15:00:00,VICTORIA DE LAS DEMOCRACIAS,19.471810,-99.164580,fraud
2,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,FUERO COMUN,2018.0,2018-12-22,15:30:00,COPILCO EL BAJO,19.337970,-99.186110,robbery with violence
3,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,FUERO COMUN,2019.0,2019-01-04,06:00:00,PANTITLAN V,19.403270,-99.059830,robbery without violence
4,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,FUERO COMUN,2019.0,2019-01-03,20:00:00,LAS AMERICAS (U HAB),19.354800,-99.063240,robbery without violence
...,...,...,...,...,...,...,...,...,...,...,...,...,...
813882,"ROBO S/V DENTRO DE NEGOCIOS, AUTOSERVICIOS, CO...",DELITO DE BAJO IMPACTO,Femenino,69.0,FISICA,FUERO COMUN,2023.0,2023-03-31,07:30:00,DEL VALLE VII,19.372612,-99.174058,robbery without violence
813883,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,Femenino,17.0,FISICA,FUERO COMUN,2023.0,2023-03-30,22:00:00,MIGUEL DE LA MADRID HURTADO,19.347189,-99.003079,domestic violence
813884,ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO,DELITO DE BAJO IMPACTO,Femenino,43.0,FISICA,FUERO COMUN,2023.0,2023-03-31,07:00:00,LOMAS DE SANTA FE II,19.369135,-99.260935,robbery without violence
813885,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,Femenino,39.0,FISICA,FUERO COMUN,2023.0,2023-03-31,10:01:00,SAN PEDRO MARTIR (PBLO),19.267265,-99.175255,robbery without violence
